Copyright (c) 2022 AIClub

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated 
documentation files (the "Software"), to deal in the Software without restriction, including without 
limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of 
the Software, and to permit persons to whom the Software is furnished to do so, subject to the following 
conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT 
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO 
EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN 
AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE 
OR OTHER DEALINGS IN THE SOFTWARE.

Follow our courses - https://www.corp.aiclub.world/courses



In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

# Please replace the brackets below with the drive location of your folder which included subfolders for images
# Sample path: /content/drive/My Drive/ImageClassification
PATH = '/content/drive/MyDrive/BrainTumorDataset/brain_tumor_dataset'


In [ ]:
def create_model(base_model, num_classes):
    import tensorflow as tf
    # Grab the last layer and add a few extra layers to it
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    # Dense layer 1
    x=tf.keras.layers.Dense(100,activation='relu', kernel_initializer=tf.keras.initializers.VarianceScaling(), use_bias=True)(x)

    # Final layer with softmax activation
    preds=tf.keras.layers.Dense(num_classes,activation='softmax', kernel_initializer=tf.keras.initializers.VarianceScaling(), use_bias=False)(x) 
    
    # Create the final model
    model=Model(inputs=base_model.input,outputs=preds)
    return model


In [ ]:
def get_optimizer(optimizer_name, learning_rate):
    # Import keras optimizers
    from tensorflow.keras.optimizers import Adam, Adadelta, Adagrad, Adamax, Ftrl, Nadam, RMSprop, SGD
    print('Selected Optimizer', optimizer_name)
    switcher = {
        'Adadelta': Adadelta(lr=learning_rate),
        'Adagrad': Adagrad(lr=learning_rate),
        'Adam': Adam(lr=learning_rate),
        'Adamax': Adamax(lr=learning_rate),
        'FTRL': Ftrl(lr=learning_rate),
        'NAdam': Nadam(lr=learning_rate),
        'RMSprop': RMSprop(lr=learning_rate),
        'Gradient Descent': SGD(lr=learning_rate)
    }
    # If optimizer_name is empty, Adam will be return as default optimizer
    return switcher.get(optimizer_name, Adam(lr=learning_rate))


In [ ]:

# Import packages needed to create a imaage classification model
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from keras.applications.resnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras import regularizers

from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.callbacks import EarlyStopping
from tensorflow import keras

# Initialize hyper params
epochs = 40 #<-- increase for higher accuracy
base_learning_rate = 0.005 #decrease for different results; use excel sheet to note down results from each change to learning rate and epochs
optimizer = 'Adam'
BATCH_SIZE = 32

IMG_SIZE = (224, 224)

# Create the data generation pipeline for training and validation
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(PATH,
                                                target_size=IMG_SIZE,
                                                color_mode='rgb',
                                                batch_size=BATCH_SIZE,
                                                class_mode='categorical',
                                                shuffle=True,
                                                subset = 'training')
validation_generator = train_datagen.flow_from_directory(PATH,
                                                target_size=IMG_SIZE,
                                                color_mode='rgb',
                                                batch_size=BATCH_SIZE,
                                                class_mode='categorical',
                                                shuffle=True,
                                                subset = 'validation')

# Download the model, valid alpha values [0.25,0.35,0.5,0.75,1]
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet', alpha=0.35)
for layer in base_model.layers:
    layer.trainable=False

# Specify the number of classes
num_classes = 2

# Create the base model
model = create_model(base_model,num_classes)

print(len(base_model.layers))

model.compile(optimizer = get_optimizer(optimizer_name=optimizer,learning_rate=base_learning_rate),loss='CategoricalCrossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=30,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

step_size_train = train_generator.n//train_generator.batch_size
history_fine = model.fit(train_generator,
                        epochs=epochs,
                        callbacks=[early_stopping_monitor],
                        validation_data = validation_generator,
                        verbose=1)


Found 184 images belonging to 2 classes.
Found 45 images belonging to 2 classes.
154
Selected Optimizer Adam


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adadelta.py:74: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adadelta, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adagrad.py:74: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adagrad, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adamax.py:90: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adamax, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/ftrl.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Ftrl, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-

Epoch 1/40
6/6 [==============================] - 8s 830ms/step - loss: 1.3085 - accuracy: 0.5217 - val_loss: 0.8345 - val_accuracy: 0.4000
Epoch 2/40
6/6 [==============================] - 4s 610ms/step - loss: 0.5618 - accuracy: 0.6739 - val_loss: 0.6286 - val_accuracy: 0.7111
Epoch 3/40
6/6 [==============================] - 4s 631ms/step - loss: 0.4848 - accuracy: 0.7554 - val_loss: 0.4735 - val_accuracy: 0.7111
Epoch 4/40
6/6 [==============================] - 4s 625ms/step - loss: 0.3734 - accuracy: 0.8370 - val_loss: 0.5606 - val_accuracy: 0.7556
Epoch 5/40
6/6 [==============================] - 4s 603ms/step - loss: 0.3200 - accuracy: 0.8696 - val_loss: 0.5079 - val_accuracy: 0.7111
Epoch 6/40
6/6 [==============================] - 4s 639ms/step - loss: 0.3016 - accuracy: 0.9076 - val_loss: 0.5969 - val_accuracy: 0.7778
Epoch 7/40
6/6 [==============================] - 4s 606ms/step - loss: 0.2922 - accuracy: 0.8696 - val_loss: 0.5844 - val_accuracy: 0.7778
Epoch 8/40
6/6 [====

In [ ]:

# Import numpy for calculating best model accuracy
import numpy as np
# Populating matrics -> accuracy & loss
acc = history_fine.history['accuracy']
val_acc = history_fine.history['val_accuracy']

loss = history_fine.history['loss']
val_loss = history_fine.history['val_loss']

print('Training Accuracy: ', acc)
print('Validation Accuracy: ', val_acc)
print('Training Loss: ', loss)
print('Validation Loss: ', val_loss)
best_model_accuracy = history_fine.history['val_accuracy'][np.argmin(history_fine.history['val_loss'])]
print('best model accuracy: ', best_model_accuracy)


Training Accuracy:  [0.52173912525177, 0.6739130616188049, 0.7554348111152649, 0.8369565010070801, 0.8695651888847351, 0.907608687877655, 0.8695651888847351, 0.907608687877655, 0.9239130616188049, 0.9021739363670349, 0.91847825050354, 0.945652186870575, 0.9728260636329651, 0.97826087474823, 0.9619565010070801, 0.95652174949646, 0.989130437374115, 0.97826087474823, 0.989130437374115, 0.9945651888847351, 0.989130437374115, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Validation Accuracy:  [0.4000000059604645, 0.7111111283302307, 0.7111111283302307, 0.7555555701255798, 0.7111111283302307, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.800000011920929, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.800000011920929, 0.800000011920929, 0.7777777910232544, 0.800000011920929, 0.800000011920929, 0.7777777910232544, 0.800000011920929, 0.7777777910232544, 0.800000011920929,

In [ ]:

# Predicting code for an image
from tensorflow.keras.preprocessing import image
# Please replace the brackets below with the location of your image which need to predict
img_path = '/content/drive/MyDrive/BrainTumorDataset/brain_tumor_test/yes/Y21.jpg'
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
img_preprocessed = preprocess_input(img_batch)
prediction = model.predict(img_preprocessed)
print(prediction)


[[0.01441134 0.9855886 ]]
